### Objective:

Suppose writing the backend for a Widget online sales application, create a base `WidgetException` class that will be used as the base class for all custom exceptions we raise from our Widget application.

---

### Functionalities:

1. Define following exceptions:

```
* Supplier exceptions
    a. Not manufactured anymore
    b. Production delayed
    c. Shipping delayed
    
* Checkout exceptions
    a. Inventory type exceptions
        - out of stock
    b. Pricing exceptions
        - invalid coupon code
        - cannot stack coupons
```

2. Implement separate internal error message and user error message
3. Implement an http status code associated to each exception type (keep it simple, use a 500 (server error) error for
everything except invalid coupon code, and cannot stack coupons, these can be 400 (bad request)
4. Implement a logging function that can be called to log the exception details, time it occurred, `TracebackException`, etc.
5. Implement a function that can be called to produce a json string containing the exception details you want to display to your user (include http status code (e.g. 400), the user error message, etc)

---

### Implementation:

In [1]:
import json
import logging
import time
import datetime
from http import HTTPStatus
import traceback

In [2]:
WIDGET_MESSAGE = 'An exception occurred in the widget!'
SUPPLIER_MESSAGE = 'Something is wrong with supplier end!'
CHECKOUT_MESSAGE = 'Error occurred whening checking out!'
MANUFACTURE_MESSAGE = 'The product is not produced anymore!'
PRODUCTION_DELAY_MESSAGE = 'The product production is going to be delayed!'
SHIPPING_DELAY_MESSAGE = 'Shipping will be delayed!'
INVENTORY_TYPE_MESSAGE = 'Inventory type is incorrect!'
PRICING_MESSAGE = 'An exception occurred during pricing process!'
OUT_OF_STOCK_MESSAGE = 'Product is currently out of stock!'
INVALID_COUPON_MESSAGE = 'The coupon code is invalid!'
CANNOT_STACK_COUPONS_MESSAGE = 'Coupons could not be stacked!'

In [3]:
class WidgetException(Exception):
    internal_message = WIDGET_MESSAGE
    http_status = HTTPStatus.INTERNAL_SERVER_ERROR
    
    def __init__(self, *args):
        super().__init__(args)
        self.args = args
        if self.args:
            self.user_message = args[0]
    
    def display(self):
        exception_details = {
            'UTC Time': str(datetime.datetime.utcnow()),
            'Type': type(self).__name__,
            'Http Status': self.http_status.value,
            'User Error Message': f'{self.http_status.phrase}: {self.user_message}',
            'Arguments': self.args[1:],
            'Traceback': list(self.traceback)
        }
        return json.dumps(exception_details)
    
    def log(self):
        logging.basicConfig(filename='./project6.log', 
                            level=logging.INFO,
                            format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                            datefmt="%Y-%m-%dT%H:%M:%S",
                            filemode='w')
        logger = logging.getLogger(name='exception_logger')
        logger.info(self.display())
       
    @property
    def traceback(self):
        return traceback.TracebackException.from_exception(self).format()

In [4]:
WidgetException('Custom message', 500, 1000).display()

'{"UTC Time": "2023-07-13 15:07:29.981345", "Type": "WidgetException", "Http Status": 500, "User Error Message": "Internal Server Error: Custom message", "Arguments": [500, 1000], "Traceback": ["WidgetException: (\'Custom message\', 500, 1000)\\n"]}'

In [5]:
class SupplierException(WidgetException):
    internal_message = SUPPLIER_MESSAGE

In [6]:
class CheckoutException(WidgetException):
    internal_message = CHECKOUT_MESSAGE

In [7]:
class NotManufacturedAnymore(SupplierException):
    internal_message = MANUFACTURE_MESSAGE

In [8]:
class ProductionDelayed(SupplierException):
    internal_message = PRODUCTION_DELAY_MESSAGE

In [9]:
class ShippingDelayed(SupplierException):
    internal_message = SHIPPING_DELAY_MESSAGE

In [10]:
class InventoryTypeException(CheckoutException):
    internal_message = INVENTORY_TYPE_MESSAGE

In [11]:
class PricingException(CheckoutException):
    internal_message = PRICING_MESSAGE

In [12]:
class OutOfStock(InventoryTypeException):
    internal_message = OUT_OF_STOCK_MESSAGE

In [13]:
class InvalidCouponCode(PricingException):
    internal_message = INVALID_COUPON_MESSAGE

In [14]:
class CannotStackCoupons(PricingException):
    internal_message = CANNOT_STACK_COUPONS_MESSAGE